In [145]:
from math import sqrt
import numpy as np
import trimesh as tm
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay as dl
import random 
from operator import truediv
from pickle import FALSE, TRUE
from traceback import print_tb
import trimesh as tm
import numpy as np
from collections import defaultdict, deque
import itertools



In [146]:


mesh = tm.load("lowlaurana.obj", file_type='obj')
print(mesh.faces)
print("-----------------")
E = mesh.edges_unique
#E = mesh.edges
V = mesh.vertices 
#V = np.matrix('1.0 1.0 1.0; 3.0 4.0 3.0; 0 0 0; -5.0 -2.0 -3.0')
#E = np.matrix('0 1; 0 2; 1 2; 1 3;2 3')

print(V)
print("-----------------")
print(E)
print("-----------------")

def collapse(n):
    for i in range(n):
    #iteramos para saber cual es el error mas bajo para juntar los vertices n veces
        lowErrorIndex = 0 
        cont = 0
        lowError = calculateLoss(E[0])
        for e in E:
            lowErrorAux = calculateLoss(e)
            if(lowErrorAux<lowError): 
                lowError = lowErrorAux
                lowErrorIndex = cont
            cont +=1
        deleteVertix(np.minimum(E[lowErrorIndex,0],E[lowErrorIndex,1]),np.maximum(E[lowErrorIndex,0],E[lowErrorIndex,1]))

def calculateLoss(e):
    x = V[e[0],0] - V[e[1],0]
    y = V[e[0],1] - V[e[1],1]
    z = V[e[0],2] - V[e[1],2]
    return sqrt(pow(x,2)+pow(y,2)+pow(z,2))

def deleteVertix(i,j):
    # i tiene que ser menor que j
    global V,E

    V[i,0] = (V[i,0] + V[j,0])/2
    V[i,1] = (V[i,1] + V[j,1])/2
    V[i,2] = (V[i,2] + V[j,2])/2
    
    V = np.delete(V,j,0)

    for e in range(len(E)):
        if (E[e,0] == i and E[e,0] == j) or (E[e,0] == j and E[e,0] == i): 
            E = np.delete(E,e,0)
            break
    
    deleted = 0
    deletedEdges = []
    nonDeletedEdges = []
    for e in range(len(E)):
        e -= deleted
        if (E[e,0] == j or E[e,1] == j):
            if E[e,0] > j: E[e,0] -= 1
            if E[e,1] > j: E[e,1] -= 1
            deletedEdges.append(E[e])
            E = np.delete(E, e, 0)
            deleted += 1
        elif (E[e,0] == i or E[e,1] == i):
            if E[e,0] > j: E[e,0] -= 1
            if E[e,1] > j: E[e,1] -= 1
            nonDeletedEdges.append(E[e])
        else:
            if E[e,0] > j: E[e,0] -= 1
            if E[e,1] > j: E[e,1] -= 1
    
    for idxD in range(len(deletedEdges)):
        dE = deletedEdges[idxD]
        posD = 1
        if dE[1] == j: posD = 0
        same = FALSE
        for idxN in range(len(nonDeletedEdges)):
            nDE = nonDeletedEdges[idxN]
            posN = 1
            if nDE[1] == i: posN = 0
            if dE[posD] == nDE[posN]:
                same = TRUE
                break
        if same == FALSE and i != dE[posD]:
            auxE = np.array([i,dE[posD]])
            E = np.vstack([E,auxE])

def facetize(edges):

    edge_lookup = defaultdict(set)
        
    #lleno el dicionario
    for a, b in edges:
        edge_lookup[a] |= {b}
        edge_lookup[b] |= {a}
        
    faces = np.matrix([[0,0,0]])
    for a in range(len(V)):
        for b in edge_lookup[a]:
            for c in edge_lookup[a]:
                if b in edge_lookup[c]:
                    faces = np.vstack([faces,sorted([a, b, c])])
    return faces


[[5045 3060 3506]
 [4811 3060 5045]
 [3179  499 1941]
 ...
 [5141  982 5175]
 [5145 5141 5183]
 [5141 5175 5183]]
-----------------
[[  3.348427   8.841011  28.817814]
 [-12.829456  13.07177   23.992275]
 [ -8.425301   3.852318  26.543104]
 ...
 [  1.989166  -7.50412   31.706457]
 [  2.47271   -7.15773   31.792255]
 [  1.980522  -7.510551  31.702866]]
-----------------
[[   8   10]
 [  11   22]
 [  23   24]
 ...
 [4312 5189]
 [5157 5189]
 [5187 5189]]
-----------------


In [ ]:
collapse(1)
nmesh = tm.Trimesh(vertices=V,faces=facetize(E))
tm.repair.fill_holes(nmesh)
#nmesh.show()
# nmesh.visual = tm.visual.ColorVisuals()

# tengo que actualizar el mesh y ya despues lo muestro
#tm.viewer.SceneViewer(mesh)